In [908]:
import numpy as np

In [909]:
from tensorflow.keras import datasets

(X_tn0, y_tn0), (X_te0, y_te0) = datasets.mnist.load_data()

In [910]:
input = X_tn0[0]
input.shape

(28, 28)

In [911]:
target = np.array(y_tn0[0]).reshape(1, -1)
target

array([[5]], dtype=uint8)

In [912]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    return self.predict - self.target

In [913]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):  
    return self.sigmoid(x) * (1 - self.sigmoid(x))

In [914]:
class PADDING:

  def padding(self, input, padding_size):
    """
    input : 입력 데이터
    padding_size : padding 연산 후의 데이터 크기
    """
    if(padding_size == 0):
      return input

    padding_matrix = np.zeros((input.shape[0] + padding_size, input.shape[1] + padding_size))

    for i in range(input.shape[0]):
      for j in range(input.shape[1]):
        padding_matrix[i + int(padding_size / 2)][j + int(padding_size / 2)] = input[i][j]
    
    return padding_matrix


In [915]:
class POOLING:
  pad = PADDING()

  pool_result = []

  def max_pooling(self, input, pooling_size):
    """
    input : 입력 데이터
    pooling_size : pooling size
    """
    self.pool_result = []
    
    pooling_matrix = self.pad.padding(input, input.shape[0] % pooling_size)

    for col in range(0, pooling_matrix.shape[0], pooling_size):
      for row in range(0, pooling_matrix.shape[1], pooling_size):
        pool_arr = []
        for pooling_col in range(pooling_size):
          for pooling_row in range(pooling_size):        
            pool_arr.append(pooling_matrix[pooling_col + col, pooling_row + row])
        self.pool_result.append(max(pool_arr))
    
    # 연산 결과를 크기에 맞게 바꿔준다.
    return np.array(self.pool_result).reshape(int(pooling_matrix.shape[0] / pooling_size), -1)

  # 풀링 결과와 매칭되는 입력값을 찾고, 해당 입력값과 연결되는 가중치에 대한 업데이트를 수행하기 위한 함수
  def matching_pooling_result(self, input, result, wight_size):
    """
    input : pooling 수행 전 입력 데이터
    result : pooling 수행 후의 데이터
    w : pooling 수행 전 입력 데이터의 결과를 만들어낸 결과 w
    pooling_size
    """
    position_arr = []
  
    # 풀링 데이터의 반복
    for pooling_col in range(result.shape[0]):
      for pooling_row in range(result.shape[1]):
        # 전체 데이터에 대한 탐색
        for col in range(pooling_col * result.shape[0], input.shape[0] - result.shape[0] - 1):
          for row in range(pooling_col * result.shape[1], input.shape[1] - result.shape[0] - 1):
            # 풀링 결과와 동일한 값을 찾는다. 그 위치의 탐색
            if(result[pooling_col, pooling_row] == input[col, row]):
              data_position = [pooling_col + col, pooling_row + row]
        
        position_arr.append(data_position)

    return position_arr



In [916]:
class MLP:
  #가중치
  weight = []

  #편향값
  bias = []
    
  # 각 층별 노드들의 출력
  node_output = []

  # 타겟, 목푯값
  target = []

  # 각 층의 델타 값의 저장
  delta = []

  # 비용 함수
  cost = cost_function()

  # 활성화 함수
  activation = activation_function()

  # 가중치 업데이트 크기
  weight_update_arr = []
  
  #순전파 계산
  def forward_cal(self, input, node_count):

    # 가중치의 임의 생성, node_count 개수만큼의 은닉(또는 출력) 노드가 존재한다.
    # (n,1) 개의 입력 값과 m 개의 노드 연결 (m, n) 크기의 가중치가 존재해야 한다.
    weight = np.random.rand(node_count, input.shape[0])

    #편향값의 임의 생성, 동일한 편향 값을 사용한다.
    bias = np.random.rand(1)

    #가중치와 노드 출력의 행렬곱연산, 편향값 덧셈
    hidden_input = weight @ input + bias
            
    #노드 입력과 활성화 함수 연산을 통한 노드 출력 계산
    # cnn 에선 0과 1 사이의 결과를 출력해야 하기 때문에 시그모이드 활성화 함수를 사용용
    output = self.activation.sigmoid(hidden_input)
            
    #노드 출력의 저장(델타 값의 계산을 위해)
    self.node_output.append(output)

    # 가중치 값 저장
    self.weight.append(weight)

    # 편향 값 저장
    self.bias.append(bias)

    return output

  def cost_function(self, target):
    self.target = target
    return self.cost.errer_squared_sum(self.predict, target)

  # 비용 함수에 대한 delta 값 계산
  def cal_delta_result(self, predict, target):  
    #출력층 노드의 변화량에 대한 오차 함수의 변화량 계산
    delta = (self.cost.diff_error_squared_sum(predict, target) * self.activation.sigmoid_diff(predict))

    self.delta.append(delta)

    return delta

  # 은닉층에 대한 delta 값 계산, 가중치 변화량이 누적된다.
  def cal_delta_hidden(self, node_input):
    # 이전층의 delta 값을 구하기 위해 
    delta = self.weight[len(self.weight) - len(self.delta)].T @ self.delta[(len(self.delta) - 1)]
    delta = delta * self.activation.sigmoid_diff(node_input)

    self.delta.append(delta)

    return delta
  
  def weight_update(self, learning_rate):
    # 연산의 편리함을 위해 뒤집어준다.
    self.delta = self.delta[::-1]
    for i in range(len(self.weight)):
      self.weight_update_arr.append(self.delta[i+1] @ self.node_output[i].T)

      self.weight[i] = self.weight[i] - self.weight_update_arr[i]



In [917]:
class CNN:

  # 필터의 개수만큼 가중치가 존재
  cnn_weight = []
  
  # 패딩
  pad = PADDING()

  # 풀링
  pool = POOLING()

  # MLP
  mlp = MLP()  

  # 데이터 크기
  data_size = 0

  # 레이어 구현
  layer_result = []

  # 현재 레이어 
  layer = 0

  # delta 값
  delta = []

  # target 값
  target = []
  
  def cal_cnn(self, input, target, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []

    # 가중치 행렬 생성
    weight_arr = []

    # 타겟값 저장
    self.target = target

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):

      # 필터 크기에 맞는 임의의 가중치 생성
      weight = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)
      
      weight_arr.append(weight)
      
      # 합성곱 연산 결과가 저장된다.
      result_arr = []

      # 합성곱 연산 수행
      for col in range(input.shape[0] - weight.shape[0] + 1):
        for row in range(input.shape[1] - weight.shape[1] + 1):
          result = []
          for w_col in range(weight.shape[0]):
            for w_row in range(weight.shape[1]):
              result.append(input[col + w_col, row + w_row ] * weight[w_col, w_row])
          result_arr.append(np.sum(result))

      # 연산 결과를 크기에 맞게 바꿔준다.
      result_arr = np.array(result_arr).reshape(input.shape[0] - weight.shape[0] + 1, -1)

      # 연산 결과의 저장 (numpy 형태)
      filter_result_arr.append(result_arr)
    
    # 가중치 행렬의 저장
    self.cnn_weight.append(weight_arr)

    # 층에다가 결과를 저장한다. 모든 필터의 결과가 저장
    self.layer_result.append(filter_result_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  def same_padding_cnn(self, input, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []
    
    # 데이터 크기 지정
    self.data_size = input.shape[0]

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):

      # 필터 크기에 맞는 임의의 가중치 생성
      weight = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)
      
      self.cnn_weight.append(weight)
      
      # 합성곱 연산 결과가 저장된다.
      result_arr = []

      # 합성곱 연산 수행
      for col in range(input.shape[0] - weight.shape[0] + 1):
        for row in range(input.shape[1] - weight.shape[1] + 1):
          result = []
          for w_col in range(weight.shape[0]):
            for w_row in range(weight.shape[1]):
              result.append(input[col + w_col, row + w_row ] * weight[w_col, w_row])
          result_arr.append(np.sum(result))

      # 연산 결과를 크기에 맞게 바꿔준다.
      result_arr = np.array(result_arr).reshape(input.shape[0] - weight.shape[0] + 1, -1)

      result_arr = self.pad.padding(result_arr, input.shape[0] - result_arr.shape[0])

      # 연산 결과의 저장
      filter_result_arr.append(result_arr)
       
    # 층에다가 결과를 저장한다.
    self.layer_result.append(filter_result_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  def pooling(self, pooling_size):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []

    # 이전 레이어의 필터 개수만큼 반복.
    for i in range(len(self.layer_result[self.layer - 1])):
      # numpy array 의 반환
      pooling_result = self.pool.max_pooling(self.layer_result[self.layer - 1][i], pooling_size)

      filter_result_arr.append(pooling_result)
    
    self.layer_result.append(filter_result_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  # 두 번째 층 이후의 cnn 연산 함수, 입력값은 이전 층의 출력이 된다.
  def layer_cnn(self, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []

    # 각 layer 별 가중치를 저장할 리스트
    weight_arr = []

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):

      # 필터 크기에 맞는 임의의 가중치 생성
      weight = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)
      
      weight_arr.append(weight)
      
      # 합성곱 연산 결과가 저장된다.
      result_arr = []

      # 합성곱 연산 수행. 이전 레이어의 필터 출력 크기에 맞게 반복이 시행된다.
      for col in range(self.layer_result[self.layer - 1][0].shape[0] - weight.shape[0] + 1):
        for row in range(self.layer_result[self.layer - 1][0].shape[1] - weight.shape[1] + 1):
          result = []
          for w_col in range(weight.shape[0]):
            for w_row in range(weight.shape[1]):
              result.append(input[col + w_col, row + w_row ] * weight[w_col, w_row])
          result_arr.append(np.sum(result))
          
      # 연산 결과를 크기에 맞게 바꿔준다.
      result_arr = np.array(result_arr).reshape(self.layer_result[self.layer - 1][0].shape[0] - weight.shape[0] + 1, -1)

      result_arr = self.pad.padding(result_arr, self.layer_result[self.layer - 1][0].shape[0] - result_arr.shape[0])

      # 연산 결과의 저장
      filter_result_arr.append(result_arr)
    
    # 층에다가 결과를 저장한다.
    self.layer_result.append(filter_result_arr)

    # 가중치의 저장
    self.cnn_weight.append(weight_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  def flatten(self):
    data = self.layer_result[self.layer - 1]
    
    data_arr = []

    # 필터의 개수만큼 반복
    for filter_count in range(len(data)):
      for filter_size_col in range(data[filter_count].shape[0]):
        for filter_size_row in range(data[filter_count].shape[1]):
          data_arr.append(data[filter_count][filter_size_col][filter_size_row])

    self.layer_result.append(np.array(data_arr).reshape(-1,1))

    self.layer = self.layer + 1

  # MLP 층의 연결, 순전파 연산

  def mlp_forward_cal(self, node_count):
    if(len(self.mlp.node_output) == 0):
      self.mlp.node_output.append(self.layer_result[self.layer - 1])

    self.layer_result.append(self.mlp.forward_cal(self.layer_result[self.layer-1], node_count))

    self.cnn_weight.append(self.mlp.weight)

    self.layer = self.layer +1   

  def cal_delta_result_layer(self):

    self.delta.append(self.mlp.cal_delta_result(self.layer_result[self.layer - 1], self.target))


  def cal_delta_hidden_layer(self):
    # 노드 입력값을 넘겨준다.
    self.delta.append(self.mlp.cal_delta_hidden(self.layer_result[self.layer - len(self.delta) - 1]))

  def cal_weight_update_mlp(self, learning_rate):
    self.mlp.weight_update(learning_rate)

  def cal_delta_pooling(self, pooling_size):
    # 커널 개수만큼의 delta 값
    kernel_delta_arr = []

    # 새로 저장되는 delta 값의 크기 계산산
    kernel_delta = np.zeros(len(self.delta[len(self.delta) - 1]))

    # 커널의 개수만큼의 반복
    print()
    for i in range(len(self.layer_result[self.layer - len(self.delta) - 1])):

      # 커널별 최댓값의 위치를 탐색한다.
      max_position = self.pool.matching_pooling_result(cnn.layer_result[self.layer - len(self.delta) - 2][i], cnn.layer_result[self.layer - len(self.delta) - 1][i], pooling_size)
      
      for j in range(len(max_position)):
        # 추가되는 값의 위치 계산
        delta_position = (i * (cnn.layer_result[self.layer - len(self.delta) - 2][i]).shape[0] ** 2)
        delta_position = delta_position + (max_position[j][0] * (cnn.layer_result[self.layer - len(self.delta) - 2][i]).shape[0]) + max_position[j][1]
        print(delta_position)

        


In [918]:
cnn = CNN()

In [919]:
cnn.target = target

In [920]:
cnn.same_padding_cnn(input, 7, 64)

In [921]:
cnn.pooling(2)

In [922]:
cnn.layer_cnn(3, 128)

In [923]:
cnn.pooling(2)

In [924]:
cnn.layer_cnn(3, 256)

In [925]:
cnn.pooling(2)

In [926]:
cnn_result = cnn.flatten()

In [927]:
cnn.mlp_forward_cal(128)

In [928]:
cnn.mlp_forward_cal(64)

In [929]:
cnn.mlp_forward_cal(10)

In [930]:
cnn.cal_delta_result_layer()

In [931]:
cnn.cal_delta_hidden_layer()

In [932]:
cnn.cal_delta_hidden_layer()

In [933]:
cnn.cal_delta_hidden_layer()

In [934]:
cnn.cal_weight_update_mlp(0.1)

In [935]:
a = cnn.pool.matching_pooling_result(cnn.layer_result[4][0], cnn.layer_result[5][0], 2)

In [936]:
cnn.layer - len(cnn.delta)

6

In [937]:
cnn.layer_result[4][0].shape

(7, 7)

In [938]:
print(len(cnn.delta[0]), len(cnn.delta[1]), len(cnn.delta[2]), len(cnn.delta[3]))

10 64 128 4096


In [939]:
cnn.delta[3].shape

(4096, 1)

In [940]:
cnn.cal_delta_pooling(2)


8
10
12
14
22
24
26
28
36
38
40
42
50
52
54
56
57
59
61
63
71
73
75
77
85
87
89
91
99
101
103
105
106
108
110
112
120
122
124
126
134
136
138
140
148
150
152
154
155
157
159
161
169
171
173
175
183
185
187
189
197
199
201
203
204
206
208
210
218
220
222
224
232
234
236
238
246
248
250
252
253
255
257
259
267
269
271
273
281
283
285
287
295
297
299
301
302
304
306
308
316
318
320
322
330
332
334
336
344
346
348
350
351
353
355
357
365
367
369
371
379
381
383
385
393
395
397
399
400
402
404
406
414
416
418
420
428
430
432
434
442
444
446
448
449
451
453
455
463
465
467
469
477
479
481
483
491
493
495
497
498
500
502
504
512
514
516
518
526
528
530
532
540
542
544
546
547
549
551
553
561
563
565
567
575
577
579
581
589
591
593
595
596
598
600
602
610
612
614
616
624
626
628
630
638
640
642
644
645
647
649
651
659
661
663
665
673
675
677
679
687
689
691
693
694
696
698
700
708
710
712
714
722
724
726
728
736
738
740
742
743
745
747
749
757
759
761
763
771
773
775
777
785
787
789
791
792
7